## Introduction to Latent Semantic Analysis

Currently, our vector space has a very **high dimensionality**, since there is a separate
dimension for every single word in the vocabulary. 

There are a couple of **small problems** with this:

* Computations can take a long time.
* The vectors are hard to visualize.

There is also **one big problem**:
* Passages of text that are made up of similar, but non-identical words, can have a dot product of zero.

With so many dimensions, there's lots of room for all of the vectors to spread out, so that they have little overlap. But we'd like for passages with similar *meaning* to have a non-zero dot product.

There are a number of ways of dealing with this problem. One is **latent semantic analysis**. 

The primary innovation in LSA is that it uses a procedure called **singular value decomposition** to squish all of those dimensions down into a smaller number of dimensions. This starts to give overlaps where there were none before.

Another __innovation typically__ used in LSA is the use of a **training corpus**. I'll try to remember to briefly mention this at the end.

In [ ]:
import numpy as np
from numpy import linalg
from IPython.core.display import Image
from sympy import *
init_printing()
np.set_printoptions(precision=3)

In [ ]:
def round_matrix(the_matrix, prec = 2):
    sh = the_matrix.shape
    if len(sh) == 1:
        for i in range(sh[0]):
            the_matrix[i] = round(the_matrix[i], prec)
    else:
        for i in range(sh[0]):
            for j in range(sh[1]):
                the_matrix[i, j] = round(the_matrix[i, j], prec)
    return the_matrix
def show_rounded_matrix(the_matrix):
    return Matrix(round_matrix(the_matrix))

I'm going to use a very simple example from: 

**Landauer, T., Peter W. Foltz, and D. Laham. “An Introduction to Latent Semantic Analysis.” *Discourse Processes* 25 (1998): 259–84.**

In [ ]:
Image(filename="images/land-data.jpg")

In [ ]:
doc_dict = {
    "c1": "Human machine interface for ABC computer applications",
    "c2": "A survey of user opinion of computer system response time",
    "c3": "The EPS user interface management system",
    "c4": "System and human system engineering testing for EPS",
    "c5": "Relation of user perceived response time to error measurement",
    "m1": "The generation of random, binary, ordered trees",
    "m2": "The intersection of graph of photos in trees",
    "m3": "Graph minors IV Widths of trees and well-quasi-ordering",
    "m4": "Graph minors A survey" 
}

## First we'll do an analysis **without** LSA

Make a termxdocument matrix in the usual way

In [ ]:
Image(filename="images/land-matrix.jpg")

In [ ]:
dnames = sorted(doc_dict.keys())

vocab = ['human', 'interface', 'computer', 'user', 'system', 'response', 'time', 'eps', 'survey', 'trees', 'graph', 'minors']

In [ ]:
for k, val in doc_dict.items():
    doc_dict[k] = val.lower().split(" ")

In [ ]:
td_matrix = np.zeros([len(vocab), len(dnames)])
i = 0
for term in vocab:
    new_row = [doc_dict[dname].count(term) for dname in dnames]
    td_matrix[i, :] = new_row
    i = i + 1
print("td_matrix = ")
show_rounded_matrix(td_matrix)

### Find the documentxdocument similarities

The first step is to normalize the document vectors, which are the columns in the above matrix.

In [ ]:
import numpy as np
def norm_vec(v):
    if np.linalg.norm(v) == 0:
        return np.zeros(len(v))
    else:
        return v / np.linalg.norm(v)

def normalize_columns(mat):
    mat_norm = np.copy(mat)
    for i in range(mat.shape[1]):
        mat_norm[:, i] = norm_vec(mat[:, i])
    return mat_norm

def normalize_rows(mat):
    mat_norm = np.copy(mat)
    for i in range(mat.shape[0]):
        mat_norm[i, :] = norm_vec(mat[i, :])
    return mat_norm

In [ ]:
td_col_norm = normalize_columns(td_matrix)

In [ ]:
show_rounded_matrix(td_col_norm)

**To get docxdoc similarities with do $A^TA$**


In [ ]:
doc_doc_comparisons = np.dot(np.transpose(td_col_norm), td_col_norm)
show_rounded_matrix(doc_doc_comparisons)

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

def matrix_heatmap(mtx, name_list):
    fig=plt.figure(figsize=(6, 6), dpi= 80, facecolor='w', edgecolor='k')
    n = len(name_list)
    x_tick_marks = np.arange(n)
    y_tick_marks = np.arange(n)
    plt.xticks(x_tick_marks, name_list, fontsize=8, rotation=90)
    plt.yticks(y_tick_marks, name_list, fontsize=8)
    plt.tick_params("x", top=True, labeltop=True, bottom=False, labelbottom=False)
    plt.imshow(mtx, norm=matplotlib.colors.LogNorm(), interpolation='nearest', cmap='YlOrBr')

In [ ]:
matrix_heatmap(doc_doc_comparisons, doc_dict.keys())

### Term x term comparisons: $AA^T$ 

**As before, we think of the rows of the matrix as representing the terms**

In [ ]:
td_row_norm = normalize_rows(td_matrix)

In [ ]:
term_term_comparisons = np.dot(td_row_norm, np.transpose(td_row_norm))
print("term x term comparisons =")
show_rounded_matrix(term_term_comparisons)

In [ ]:
matrix_heatmap(term_term_comparisons, vocab)

### Application: Search for documents that match a query

query = "human interface"

In [ ]:
q = "human interface"

First make a vector for this query and normalize it.

In [ ]:
q_vec = np.array([q.count(term) for term in vocab], float)
q_vec_norm = norm_vec(q_vec)
print(q_vec_norm)

Then compare this vector to each of the documents

In [ ]:
dot_products = np.dot(td_col_norm.transpose(), q_vec_norm)
for i in range(len(dot_products)):
    print(dnames[i] + ": " + str(round(dot_products[i], 3)))

## Now let's do some LSA

### Using the SVD to get word vectors

The of LSA is done by the singular value decomposition.

SVD is a way of constructing an approximation to a matrix. For our purposes, the key thing to know is that SVD takes a termxdocument matrix and factors it into three matrices. We're going to make use of just one of those matrices.

$\huge {A=T_{txn}S_{nxn}(D_{dxn})^T}$

$\large n=min(t,d)$

In [ ]:
Image(filename="images/lsiexpanded2.jpg")

`Numpy` will just do the svd for us.

We have to decide how many dimensions we want to keep

In [ ]:
dims = 2
T, S, Dt = linalg.svd(td_matrix, full_matrices = False)
T_reduced = T[:, 0:dims]
Dt_reduced = Dt[0:dims, :]
S_reduced = np.diag(S)[0:dims, 0:dims]

###  Using the new word vectors to get document vectors

Here's how we're going to think of the key maneuver here.
We're going to think of the rows in `T_reduced` as being a vector for each of the words in our vocabulary.

In [ ]:
show_rounded_matrix(T_reduced)

Then, to find the vector for a document, we add the vectors corresponding to each of the words that appear. This will, in this case, give us a two dimensional vector for any document.

For example, here's the unreduced vector for c1

In [ ]:
c1_vector = td_matrix[:, 0]
print(c1_vector)

To find the squished vector for c1, we add up the word vectors for the first three words in our vocabulary.

Here I won't normalize the word vectors first. I'm not sure what's the right thing to do here.

In [ ]:
T_reduced[0] + T_reduced[1] + T_reduced[2]

The more efficient way to write this is:

In [ ]:
np.dot(T_reduced.transpose(), c1_vector)

### Plotting Vectors
Since everything is in 2 dimensions, we can make plots.
First, we plot the vectors for the words. Here they aren't normalized:

#### Plot the word vectors

Here they aren't normalized:

In [ ]:
from matplotlib import pyplot
pyplot.cla
pyplot.scatter(T_reduced[:, 0], T_reduced[:, 1])
for i in range(len(vocab)):
    pyplot.annotate(vocab[i], T_reduced[i])

#### Plot the document vectors

Now do the documents. First we find the squished vectors for each document, then plot them.

In [ ]:
dmatrix = np.dot(T_reduced.transpose(), td_matrix)
show_rounded_matrix(dmatrix)

In [ ]:
pyplot.scatter(dmatrix[0], dmatrix[1])
dmatrix_trans = np.transpose(dmatrix)
for i in range(len(dnames)):
    pyplot.annotate(dnames[i], dmatrix_trans[i])

We should probably look at these normalized

In [ ]:
dmatrix_normed = normalize_columns(dmatrix)
show_rounded_matrix(dmatrix_normed)

In [ ]:
pyplot.scatter(dmatrix_normed[0], dmatrix_normed[1])
dmatrix_normed_transpose = np.transpose(dmatrix_normed)
for i in range(len(dnames)):
    pyplot.annotate(dnames[i], dmatrix_normed_transpose[i])

#### Documentxdocument similarities:

In [ ]:
doc_doc_matrix = np.dot(dmatrix_normed.transpose(), dmatrix_normed)
show_rounded_matrix(doc_doc_matrix)

In [ ]:
matrix_heatmap(doc_doc_matrix, doc_dict.keys())

#### Wordxword similarities

In [ ]:
T_reduced_normed = normalize_rows(T_reduced)
term_term_matrix = np.dot(T_reduced_normed, T_reduced_normed.transpose())
show_rounded_matrix(term_term_matrix)

In [ ]:
matrix_heatmap(term_term_matrix, vocab)

#### Folding in a search query

To fold in a query, we find its vector just like any doc:**
$\large q' = T^Tq$

In [ ]:
print(q_vec)

In [ ]:
q_folded = np.dot(T_reduced.transpose(), q_vec.transpose())

In [ ]:
print(q_folded)

We can make a plot of the various documents and see where the query vector fits

In [ ]:
pyplot.scatter(Dt_reduced[0], Dt_reduced[1])
D_reduced = np.transpose(Dt_reduced)
for i in range(len(dnames)):
    pyplot.annotate(dnames[i], D_reduced[i])
pyplot.scatter(q_folded[0], q_folded[1])

In [ ]:
q_folded_norm = norm_vec(q_folded)
pyplot.scatter(dmatrix_normed[0], dmatrix_normed[1])
dmatrix_normed_transpose = np.transpose(dmatrix_normed)
for i in range(len(dnames)):
    pyplot.annotate(dnames[i], dmatrix_normed_transpose[i])
pyplot.scatter(q_folded_norm[0], q_folded_norm[1])

In [ ]:
matches = list(np.dot(q_folded_norm, dmatrix_normed))
for i in range(len(cosines)):
    print(dnames[i] + ": " + str(round(matches[i], 3)))

**This is what they were without LSI**

* c1: 0.816
* c2: 0.0
* c3: 0.354
* c4: 0.289
* c5: 0.0
* m1: 0.0
* m2: 0.0
* m3: 0.0
* m4: 0.0

## Finally, about the use of a training corpus

In the above procedure, we first found vectors for words in our new space. Then we used those word vectors to get the vector for any document. To do this we used the same documents that we wished to examine in order to find the word vectors.

An alternative is to use a *training corpus* to find the word vectors. 

Suppose that we have a relatively small number of short student responses about global warming, and we wanted to compare those responses to each other or to model answers. We could go to the internet and build a training corpus consisting of hundreds or thousands of documents about global warming. Then we could "learn" the word vectors from that training corpus. In essence, this would allow us to learn the similarities between words, which we could use to help us with the student essays.

So, for example, using the training corpus, we could discover word vectors for "heat" and "warmth" that were similar. Then, when creating vectors for the student responses, heat and warmth would make similar contributions to that tvector.